# Modelo predictivo de devoluciones (item-level)

En este notebook se construye y entrena un **modelo supervisado de clasificación binaria** cuyo objetivo es **predecir si un ítem será devuelto (`devuelto = 1`) o no**, utilizando información de producto, cliente, ticket, precio, contexto temporal y comportamiento histórico.

El modelo opera **a nivel de ítem**, no a nivel de pedido ni de cliente, lo que permite capturar patrones finos como:
- desajustes de talla,
- historial individual de devoluciones,
- comportamiento específico por producto,
- diferencias estructurales entre canal online y canal físico.

El dataset utilizado procede de la tabla `base_modelo_devoluciones`, construida previamente a partir de todo el pipeline de generación y enriquecimiento de datos.

---

## 1) Carga de datos y validaciones iniciales

Los datos se cargan desde una base de datos SQLite (`database/mi_base.db`), lo que garantiza:
- persistencia del pipeline,
- trazabilidad de los experimentos,
- independencia del notebook respecto a ficheros intermedios.

Antes de entrenar cualquier modelo se realizan **chequeos básicos**:
- número total de filas,
- tasa global de devolución,
- tasa de devolución por canal (online vs físico).

Estos chequeos permiten validar que:
- la variable objetivo está correctamente generada,
- existe un gap claro entre canales (online > físico),
- el dataset es coherente con los supuestos de negocio.

Resultados observados:
- Tasa global de devolución ≈ 0.30
- Online ≈ 0.32
- Físico ≈ 0.23

---

## 2) Ingeniería de variables (Feature Engineering)

La función `feature_engineering()` transforma la tabla base en un dataset listo para modelar.  
Las transformaciones se agrupan en varios bloques conceptuales.

---

### 2.1 Variables temporales y de cliente

A partir de la fecha de compra y las fechas de alta del cliente se construyen:

- `anio_compra`
- `mes_compra`
- `temporada_compra` (SS / FW)
- `edad_en_compra`
- `antiguedad_cliente_dias`

Estas variables permiten capturar:
- estacionalidad en las devoluciones,
- diferencias por cohortes de edad,
- comportamiento según antigüedad del cliente.

---

### 2.2 Variables económicas y promocionales

Se generan variables relacionadas con precio y margen:

- `en_promocion`
- `margen_relativo` (margen / precio_neto)

Estas variables ayudan a modelar:
- compras impulsivas,
- mayor propensión a devolución en campañas promocionales,
- sensibilidad del cliente al precio.

---

### 2.3 Inferencia de talla ideal del cliente

Se estima una **talla ideal teórica** para cada cliente a partir de sus características físicas.

#### Ropa
- Se definen rangos de altura y peso por talla (XS–XL).
- Para cada cliente se calcula una distancia normalizada a cada talla.
- Se asigna como talla ideal aquella con menor distancia.

#### Calzado
- La talla ideal se infiere únicamente a partir de la altura.
- Se selecciona la talla cuyo rango central está más próximo.

Estas tallas ideales no son visibles para el cliente, pero sirven como referencia objetiva para medir errores de ajuste.

---

### 2.4 Desajuste de talla (feature clave)

A partir de la talla comprada y la talla ideal se construyen:

- `desajuste_talla`
- `desajuste_talla_abs`

Este desajuste es uno de los **drivers más importantes** del modelo, especialmente en:
- ropa,
- calzado,
- tallas extremas.

Adicionalmente se crea la variable:
- `talla_extrema` (XS, XL, 39, 45).

---

### 2.5 Historial del cliente

Ordenando las compras cronológicamente, se generan variables acumuladas:

- `compras_previas_cliente`
- `devoluciones_previas_cliente`
- `ratio_devoluciones_previas_cliente`

Estas variables capturan:
- clientes sistemáticamente propensos a devolver,
- aprendizaje del cliente tras experiencias previas,
- patrones persistentes de comportamiento.

---

### 2.6 Historial del producto

De forma análoga, a nivel producto:

- `ventas_previas_producto`
- `devoluciones_previas_producto`
- `ratio_devoluciones_previas_producto`

Esto permite identificar:
- productos con problemas estructurales de fit,
- referencias con fricción logística elevada,
- artículos con tasas de devolución anómalas.

---

### 2.7 Precio relativo por categoría

Se calcula la variable:

- `precio_rel_cat = precio_neto / precio_medio_categoria`

Esto permite capturar:
- productos significativamente más caros o baratos dentro de su categoría,
- expectativas del cliente respecto al producto.

---

## 3) Split de entrenamiento y test

Se utiliza un **split temporal**, no aleatorio:

- 75% de las observaciones más antiguas → train
- 25% más reciente → test

Configuración:
- `mode = "temporal"`
- `test_size = 0.25`
- `date_col = "fecha_compra"`

Este enfoque:
- evita leakage temporal,
- simula un escenario real de producción,
- penaliza modelos que solo memorizan patrones históricos.

---

## 4) Construcción de matrices X / y

### 4.1 Selección de variables

Se eliminan columnas no predictivas o que inducen leakage:
- identificadores,
- fechas originales,
- variables objetivo auxiliares,
- tallas ideales explícitas.

---

### 4.2 Tratamiento de valores nulos

- Se crean indicadores `missing_*` para variables sensibles (altura, peso, edad, etc.).
- Variables numéricas → imputación por mediana (calculada solo en train).
- Valores infinitos o residuales → sustituidos por 0.0.

---

### 4.3 Variables categóricas

- Normalización de texto (minúsculas, sin acentos).
- One-hot encoding.
- Alineación exacta de columnas entre train y test.

El resultado es una matriz estable y compatible con modelos tree-based y lineales.

---

## 5) Persistencia del dataset procesado

Para garantizar reproducibilidad se guardan los siguientes artefactos:

- `X_train.parquet`, `X_test.parquet`
- `y_train.parquet`, `y_test.parquet`
- índices de train/test
- metadata del preprocesado:
  - columnas finales,
  - medianas usadas,
  - columnas categóricas,
  - configuración del split.

Todo queda almacenado en:
data/processed/devoluciones/

---

## 6) Modelos evaluados

Se entrenan varios modelos base para comparación:

- Random Forest
- Extra Trees
- HistGradientBoosting
- Logistic Regression
- SGDClassifier (log-loss)
- XGBoost (modelo final)

Todos los modelos utilizan:
- clases balanceadas,
- evaluación consistente,
- métricas centradas en la clase positiva.

---

## 7) Modelo final: XGBoost

### 7.1 Estrategia de entrenamiento

El modelo final (`xgb_final`) se entrena en dos fases:

1. **Búsqueda aleatoria de hiperparámetros**
   - 12 iteraciones
   - early stopping con métrica PR-AUC
   - selección del mejor conjunto de parámetros

2. **Entrenamiento final**
   - nuevo split interno
   - número de rondas ajustado a la mejor iteración
   - early stopping para estabilizar el modelo

Configuración clave:
- `objective = binary:logistic`
- `eval_metric = aucpr`
- `tree_method = hist`

---

### 7.2 Resultados finales en test

Evaluación sobre el conjunto de test temporal:

- **ROC-AUC (test)**: 0.698
- **PR-AUC (test)**: 0.569

Con umbral 0.5:
- Precision (positiva): 0.465
- Recall (positiva): 0.601
- F1 (positiva): 0.524

Mejor umbral optimizando F1:
- Threshold ≈ 0.46
- F1 ≈ 0.527
- Recall ≈ 0.69

Esto permite adaptar el modelo a distintos objetivos operativos:
- priorizar recall (detección temprana de devoluciones),
- o priorizar precisión (control de costes).

---

## 8) Artefactos generados

### Modelos
modelos/devoluciones/xgb_final.json

### Métricas y tracking
modelos/devoluciones/leaderboard.csv
modelos/devoluciones/saved_models.json
modelos/devoluciones/metadata/

### Datos procesados
data/processed/devoluciones/

---

## 9) Conclusión

El modelo final:
- captura correctamente el gap entre canal online y físico,
- incorpora información física, histórica y económica,
- generaliza razonablemente bien en un split temporal exigente,
- es directamente utilizable en un entorno productivo.

El pipeline completo es **reproducible**, **auditable** y **extensible** para:
- tuning adicional,
- análisis de interpretabilidad,
- simulaciones de impacto económico.



In [1]:
from __future__ import annotations

import json
import sqlite3
import time
import unicodedata
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import joblib
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    classification_report,
    precision_recall_curve,
)
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier


DB_PATH = "database/mi_base.db"
TABLE_NAME = "base_modelo_devoluciones"

OUT_DIR = Path("modelos") / "devoluciones"
OUT_DIR.mkdir(parents=True, exist_ok=True)
(OUT_DIR / "metadata").mkdir(exist_ok=True)

DATA_DIR = Path("data") / "processed" / "devoluciones"
DATA_DIR.mkdir(parents=True, exist_ok=True)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)


@dataclass
class SplitConfig:
    mode: str = "temporal"
    test_size: float = 0.25
    date_col: str = "fecha_compra"


def dump_json(obj, path: Path) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    with open(path, "w", encoding="utf-8") as f:
        json.dump(obj, f, ensure_ascii=False, indent=2)


def normalize_text(s):
    if pd.isna(s):
        return s
    s = unicodedata.normalize("NFKD", str(s)).encode("ascii", "ignore").decode("ascii")
    return s.lower().strip()


In [2]:
def compute_threshold_best_f1(y_true: np.ndarray, y_proba: np.ndarray) -> Tuple[float, float]:
    prec, rec, thr = precision_recall_curve(y_true, y_proba)
    f1 = 2 * (prec * rec) / (prec + rec + 1e-9)

    if len(thr) == 0:
        return 0.5, float(f1.max()) if len(f1) else 0.0

    idx = int(np.argmax(f1[:-1]))
    return float(thr[idx]), float(f1[idx])


def eval_binary_classifier(name: str, y_true: np.ndarray, y_proba: np.ndarray) -> Dict:
    roc = float(roc_auc_score(y_true, y_proba))
    pr = float(average_precision_score(y_true, y_proba))

    y_pred_05 = (y_proba >= 0.5).astype(int)
    rep_05 = classification_report(y_true, y_pred_05, output_dict=True, zero_division=0)

    best_thr, _ = compute_threshold_best_f1(y_true, y_proba)
    y_pred_best = (y_proba >= best_thr).astype(int)
    rep_best = classification_report(y_true, y_pred_best, output_dict=True, zero_division=0)

    return {
        "model": name,
        "roc_auc_test": roc,
        "pr_auc_test": pr,
        "f1_pos@0.5": float(rep_05["1"]["f1-score"]),
        "prec_pos@0.5": float(rep_05["1"]["precision"]),
        "rec_pos@0.5": float(rep_05["1"]["recall"]),
        "best_thr_f1": float(best_thr),
        "best_f1_pos": float(rep_best["1"]["f1-score"]),
        "best_prec_pos": float(rep_best["1"]["precision"]),
        "best_rec_pos": float(rep_best["1"]["recall"]),
        "support_pos": int(rep_best["1"]["support"]),
    }


In [3]:
def save_sklearn_model(model, name: str) -> str:
    path = OUT_DIR / f"{name}.joblib"
    joblib.dump(model, path)
    return str(path)


def save_xgb_booster(booster: xgb.Booster, name: str) -> str:
    path = OUT_DIR / f"{name}.json"
    booster.save_model(str(path))
    return str(path)


In [4]:
def load_data() -> pd.DataFrame:
    con = sqlite3.connect(DB_PATH)
    try:
        df = pd.read_sql_query(f"SELECT * FROM {TABLE_NAME}", con)
    finally:
        con.close()
    return df


def basic_checks(df: pd.DataFrame) -> None:
    print("Filas:", len(df))
    print("Tasa devolucion global:", round(float(df["devuelto"].mean()), 4))

    m_fisico = df["canal"].astype(str).str.lower().isin(["fisico", "físico", "tienda", "store"])
    print("Tasa devolucion online:", round(float(df.loc[~m_fisico, "devuelto"].mean()), 4))
    print("Tasa devolucion fisico:", round(float(df.loc[m_fisico, "devuelto"].mean()), 4))


In [5]:
def feature_engineering(df_in: pd.DataFrame) -> pd.DataFrame:
    df = df_in.copy()
    df.columns = df.columns.str.strip()

    df["fecha_compra"] = pd.to_datetime(df["fecha_compra"], errors="coerce")
    df["fecha_primer_compra"] = pd.to_datetime(df["fecha_primer_compra"], errors="coerce")
    df["fecha_ultima_compra"] = pd.to_datetime(df["fecha_ultima_compra"], errors="coerce")

    df["anio_compra"] = df["fecha_compra"].dt.year
    df["mes_compra"] = df["fecha_compra"].dt.month

    def temporada_from_mes(m):
        if pd.isna(m):
            return "NA"
        m = int(m)
        if m in [3, 4, 5, 6, 7, 8]:
            return "SS"
        if m in [9, 10, 11, 12, 1, 2]:
            return "FW"
        return "NA"

    df["temporada_compra"] = df["mes_compra"].apply(temporada_from_mes)

    df["edad_en_compra"] = df["anio_compra"] - df["anio_nacimiento"]
    df["antiguedad_cliente_dias"] = (df["fecha_compra"] - df["fecha_primer_compra"]).dt.days

    df["en_promocion"] = df["promotion_id"].notna().astype("int8")

    df["margen_relativo"] = np.where(
        df["precio_neto"] > 0,
        df["margen"] / df["precio_neto"],
        0.0
    )

    ROPA_RANGES = {
        "XS": {"h": (160, 168), "w": (50, 62)},
        "S":  {"h": (165, 174), "w": (58, 70)},
        "M":  {"h": (170, 180), "w": (65, 82)},
        "L":  {"h": (175, 185), "w": (75, 95)},
        "XL": {"h": (180, 200), "w": (90, 120)},
    }

    CALZADO_RANGES = {
        39: (160, 170),
        40: (165, 175),
        41: (168, 178),
        42: (170, 182),
        43: (172, 185),
        44: (175, 188),
        45: (178, 195),
    }

    def infer_talla_ideal_ropa(h, w):
        if pd.isna(h) or pd.isna(w):
            return np.nan
        best_talla = None
        best_score = None
        for talla, ranges in ROPA_RANGES.items():
            h_low, h_high = ranges["h"]
            w_low, w_high = ranges["w"]
            h_mid = (h_low + h_high) / 2
            w_mid = (w_low + w_high) / 2
            dh = (h - h_mid) / (h_high - h_low)
            dw = (w - w_mid) / (w_high - w_low)
            score = dh**2 * 0.7 + dw**2 * 0.3
            if (best_score is None) or (score < best_score):
                best_score = score
                best_talla = talla
        return best_talla

    def infer_talla_ideal_calzado(h):
        if pd.isna(h):
            return np.nan
        best_talla = None
        best_score = None
        for talla, (h_low, h_high) in CALZADO_RANGES.items():
            h_mid = (h_low + h_high) / 2
            score = (h - h_mid) ** 2
            if (best_score is None) or (score < best_score):
                best_score = score
                best_talla = talla
        return str(best_talla) if best_talla is not None else np.nan

    clientes = (
        df.groupby("customer_id", as_index=False)[["altura_cm", "peso_kg"]]
          .first()
    )

    clientes["talla_ideal_ropa"] = [
        infer_talla_ideal_ropa(h, w)
        for h, w in zip(clientes["altura_cm"], clientes["peso_kg"])
    ]

    clientes["talla_ideal_calzado"] = [
        infer_talla_ideal_calzado(h)
        for h in clientes["altura_cm"]
    ]

    df = df.merge(
        clientes[["customer_id", "talla_ideal_ropa", "talla_ideal_calzado"]],
        on="customer_id",
        how="left"
    )

    ROPA_CATS_LOWER = {"camiseta", "sudadera", "pantalon", "abrigo", "camisa"}
    CALZADO_CAT_LOWER = "calzado"

    cat_lower = df["categoria"].astype(str).str.lower()
    talla_str = df["talla"].astype(str)

    tallas_ropa_orden = ["XS", "S", "M", "L", "XL"]
    talla_idx_map = {t: i for i, t in enumerate(tallas_ropa_orden)}

    df["desajuste_talla"] = np.nan

    mask_ropa = cat_lower.isin(ROPA_CATS_LOWER)
    talla_idx = talla_str.map(talla_idx_map)
    talla_ideal_idx = df["talla_ideal_ropa"].map(talla_idx_map)

    mask_ropa_valid = mask_ropa & talla_idx.notna() & talla_ideal_idx.notna()
    df.loc[mask_ropa_valid, "desajuste_talla"] = (
        talla_idx[mask_ropa_valid].astype(float) - talla_ideal_idx[mask_ropa_valid].astype(float)
    )

    mask_calzado = (cat_lower == CALZADO_CAT_LOWER)
    talla_num = pd.to_numeric(df["talla"], errors="coerce")
    talla_ideal_num = pd.to_numeric(df["talla_ideal_calzado"], errors="coerce")

    mask_calzado_valid = mask_calzado & talla_num.notna() & talla_ideal_num.notna()
    df.loc[mask_calzado_valid, "desajuste_talla"] = (
        talla_num[mask_calzado_valid] - talla_ideal_num[mask_calzado_valid]
    )

    df["desajuste_talla_abs"] = df["desajuste_talla"].abs()

    tallas_extremas = {"XS", "XL", "39", "45"}
    df["talla_extrema"] = df["talla"].astype(str).isin(tallas_extremas).astype("int8")

    df["bmi"] = np.where(
        (df["altura_cm"].notna()) & (df["altura_cm"] > 0),
        df["peso_kg"] / (df["altura_cm"] / 100.0) ** 2,
        np.nan
    )

    df = df.sort_values(["fecha_compra", "customer_id", "ticket_id", "item_id"]).reset_index(drop=True)

    df["compras_previas_cliente"] = df.groupby("customer_id").cumcount()
    df["devoluciones_previas_cliente"] = df.groupby("customer_id")["devuelto"].cumsum() - df["devuelto"]
    df["ratio_devoluciones_previas_cliente"] = np.where(
        df["compras_previas_cliente"] > 0,
        df["devoluciones_previas_cliente"] / df["compras_previas_cliente"],
        0.0
    )

    df["id_producto"] = df["id_producto"].fillna("UNKNOWN_PROD")
    df["ventas_previas_producto"] = df.groupby("id_producto").cumcount()
    df["devoluciones_previas_producto"] = df.groupby("id_producto")["devuelto"].cumsum() - df["devuelto"]
    df["ratio_devoluciones_previas_producto"] = np.where(
        df["ventas_previas_producto"] > 0,
        df["devoluciones_previas_producto"] / df["ventas_previas_producto"],
        0.0
    )

    precio_med_cat = df.groupby("categoria")["precio_neto"].transform("mean")
    df["precio_rel_cat"] = np.where(
        precio_med_cat > 0,
        df["precio_neto"] / precio_med_cat,
        1.0
    )

    return df


In [6]:
def split_train_test(df_fe: pd.DataFrame, cfg: SplitConfig) -> Tuple[pd.DataFrame, pd.DataFrame]:
    if cfg.mode == "random":
        df_tr, df_te = train_test_split(
            df_fe,
            test_size=cfg.test_size,
            random_state=RANDOM_SEED,
            stratify=df_fe["devuelto"].astype("int8")
        )
        return df_tr, df_te

    if cfg.mode == "temporal":
        df_sorted = df_fe.sort_values([cfg.date_col, "ticket_id", "item_id"]).reset_index(drop=True)
        cut_idx = int((1.0 - cfg.test_size) * len(df_sorted))
        df_tr = df_sorted.iloc[:cut_idx].copy()
        df_te = df_sorted.iloc[cut_idx:].copy()
        return df_tr, df_te

    raise ValueError(f"Split mode no soportado: {cfg.mode}")


In [7]:
def build_xy_from_frames(
    df_train: pd.DataFrame,
    df_test: pd.DataFrame
) -> Tuple[pd.DataFrame, pd.Series, pd.DataFrame, pd.Series, Dict]:

    y_train = df_train["devuelto"].astype("int8")
    y_test = df_test["devuelto"].astype("int8")

    cols_to_drop = [
        "item_id", "ticket_id", "customer_id", "sku", "store_id", "id_producto",
        "fecha_compra", "fecha_primer_compra", "fecha_ultima_compra",
        "anio_nacimiento", "edad_alta",
        "devuelto",
        "talla_ideal_ropa", "talla_ideal_calzado",
        "promotion_id",
    ]

    X_train = df_train.drop(columns=[c for c in cols_to_drop if c in df_train.columns]).copy()
    X_test = df_test.drop(columns=[c for c in cols_to_drop if c in df_test.columns]).copy()

    na_flag_cols = [
        "provincia_cliente",
        "comunidad",
        "altura_cm",
        "peso_kg",
        "edad_en_compra",
        "antiguedad_cliente_dias",
        "desajuste_talla",
        "bmi",
    ]
    for col in na_flag_cols:
        if col in X_train.columns:
            X_train[f"missing_{col}"] = X_train[col].isna().astype("int8")
            X_test[f"missing_{col}"] = X_test[col].isna().astype("int8")

    cat_cols = [
        "canal",
        "provincia_tienda",
        "provincia_cliente",
        "comunidad",
        "categoria",
        "color",
        "talla",
        "temporada_compra",
    ]
    for col in cat_cols:
        if col in X_train.columns:
            X_train[col] = X_train[col].fillna("UNKNOWN").apply(normalize_text)
            X_test[col] = X_test[col].fillna("UNKNOWN").apply(normalize_text)

    num_cols_to_impute = [
        "altura_cm",
        "peso_kg",
        "edad_en_compra",
        "antiguedad_cliente_dias",
        "descuento",
        "precio_neto",
        "coste_bruto",
        "margen",
        "margen_relativo",
        "compras_previas_cliente",
        "devoluciones_previas_cliente",
        "ratio_devoluciones_previas_cliente",
        "ventas_previas_producto",
        "devoluciones_previas_producto",
        "ratio_devoluciones_previas_producto",
        "desajuste_talla",
        "desajuste_talla_abs",
        "bmi",
        "precio_rel_cat",
    ]

    medians = {}
    for col in num_cols_to_impute:
        if col in X_train.columns:
            med = float(pd.to_numeric(X_train[col], errors="coerce").median())
            medians[col] = med
            X_train[col] = pd.to_numeric(X_train[col], errors="coerce").fillna(med)
            X_test[col] = pd.to_numeric(X_test[col], errors="coerce").fillna(med)

    X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0.0)
    X_test = X_test.replace([np.inf, -np.inf], np.nan).fillna(0.0)

    used_cat_cols = [c for c in cat_cols if c in X_train.columns]
    X_train_enc = pd.get_dummies(X_train, columns=used_cat_cols, drop_first=True)
    X_test_enc = pd.get_dummies(X_test, columns=used_cat_cols, drop_first=True)

    x_encoded_columns = list(X_train_enc.columns)
    X_test_enc = X_test_enc.reindex(columns=x_encoded_columns, fill_value=0)

    base_cols = [
        "descuento",
        "precio_neto",
        "coste_bruto",
        "margen",
        "n_pedidos",
        "n_items_comprados",
        "altura_cm",
        "peso_kg",
        "anio_compra",
        "mes_compra",
        "edad_en_compra",
        "antiguedad_cliente_dias",
        "en_promocion",
        "margen_relativo",
        "desajuste_talla",
        "desajuste_talla_abs",
        "talla_extrema",
        "bmi",
        "compras_previas_cliente",
        "devoluciones_previas_cliente",
        "ratio_devoluciones_previas_cliente",
        "ventas_previas_producto",
        "devoluciones_previas_producto",
        "ratio_devoluciones_previas_producto",
        "precio_rel_cat",
        "missing_provincia_cliente",
        "missing_comunidad",
        "missing_altura_cm",
        "missing_peso_kg",
        "missing_edad_en_compra",
        "missing_antiguedad_cliente_dias",
        "missing_desajuste_talla",
        "missing_bmi",
    ]

    extra_prefixes = ["canal_", "comunidad_", "categoria_", "color_", "talla_", "temporada_compra_"]
    extra_cols = [c for c in x_encoded_columns if any(c.startswith(p) for p in extra_prefixes)]

    base_cols_present = [c for c in base_cols if c in x_encoded_columns]
    cols_final = list(dict.fromkeys(base_cols_present + extra_cols))

    X_train_final = X_train_enc[cols_final].copy()
    X_test_final = X_test_enc[cols_final].copy()

    prep_meta = {
        "x_encoded_columns": x_encoded_columns,
        "cols_final": cols_final,
        "medians": medians,
        "cat_cols": used_cat_cols,
        "note": "Medians calculadas en train y aplicadas a test. One-hot alineado por columnas de train.",
    }

    return X_train_final, y_train, X_test_final, y_test, prep_meta


In [8]:
def save_processed_data(
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series,
    df_train: pd.DataFrame,
    df_test: pd.DataFrame,
    prep_meta: Dict,
    split_cfg: SplitConfig
) -> None:

    DATA_DIR.mkdir(parents=True, exist_ok=True)

    X_train.to_parquet(DATA_DIR / "X_train.parquet", index=False)
    X_test.to_parquet(DATA_DIR / "X_test.parquet", index=False)
    y_train.to_frame("devuelto").to_parquet(DATA_DIR / "y_train.parquet", index=False)
    y_test.to_frame("devuelto").to_parquet(DATA_DIR / "y_test.parquet", index=False)

    keep_cols = ["fecha_compra", "ticket_id", "item_id", "customer_id", "devuelto"]
    keep_cols_train = [c for c in keep_cols if c in df_train.columns]
    keep_cols_test = [c for c in keep_cols if c in df_test.columns]

    df_train[keep_cols_train].to_parquet(DATA_DIR / "train_index.parquet", index=False)
    df_test[keep_cols_test].to_parquet(DATA_DIR / "test_index.parquet", index=False)

    dump_json(prep_meta, OUT_DIR / "metadata" / "preprocess_meta.json")
    dump_json(prep_meta["cols_final"], OUT_DIR / "metadata" / "cols_final.json")
    dump_json(prep_meta["x_encoded_columns"], OUT_DIR / "metadata" / "x_encoded_columns.json")

    split_meta = {
        "mode": split_cfg.mode,
        "test_size": split_cfg.test_size,
        "date_col": split_cfg.date_col,
        "n_train": int(len(df_train)),
        "n_test": int(len(df_test)),
        "target_rate_train": float(df_train["devuelto"].mean()),
        "target_rate_test": float(df_test["devuelto"].mean()),
        "min_date_train": str(pd.to_datetime(df_train[split_cfg.date_col]).min()) if split_cfg.date_col in df_train.columns else None,
        "max_date_train": str(pd.to_datetime(df_train[split_cfg.date_col]).max()) if split_cfg.date_col in df_train.columns else None,
        "min_date_test": str(pd.to_datetime(df_test[split_cfg.date_col]).min()) if split_cfg.date_col in df_test.columns else None,
        "max_date_test": str(pd.to_datetime(df_test[split_cfg.date_col]).max()) if split_cfg.date_col in df_test.columns else None,
    }
    dump_json(split_meta, OUT_DIR / "metadata" / "split_meta.json")


In [9]:
split_cfg = SplitConfig(mode="temporal", test_size=0.25, date_col="fecha_compra")

df = load_data()
basic_checks(df)

df_fe = feature_engineering(df)
df_train, df_test = split_train_test(df_fe, split_cfg)

X_train, y_train, X_test, y_test, prep_meta = build_xy_from_frames(df_train, df_test)

print("Voy a guardar datos procesados en:", DATA_DIR.resolve())
save_processed_data(X_train, y_train, X_test, y_test, df_train, df_test, prep_meta, split_cfg)
print("Datos procesados guardados")


Filas: 905445
Tasa devolucion global: 0.2988
Tasa devolucion online: 0.3196
Tasa devolucion fisico: 0.2344
Voy a guardar datos procesados en: C:\Users\PEDRO\Desktop\ropa\data\processed\devoluciones
Datos procesados guardados


In [10]:
print("Archivos en data/processed:")
for p in sorted(DATA_DIR.glob("*")):
    print("-", p.name)


Archivos en data/processed:
- test_index.parquet
- train_index.parquet
- X_test.parquet
- X_train.parquet
- y_test.parquet
- y_train.parquet


In [11]:
def build_models():
    models = []

    models.append(("rf", RandomForestClassifier(
        n_estimators=600,
        max_depth=18,
        min_samples_leaf=200,
        min_samples_split=400,
        max_features="sqrt",
        class_weight="balanced_subsample",
        n_jobs=1,
        random_state=RANDOM_SEED
    )))

    models.append(("extratrees", ExtraTreesClassifier(
        n_estimators=1400,
        max_depth=None,
        min_samples_leaf=60,
        min_samples_split=300,
        max_features="sqrt",
        bootstrap=False,
        class_weight="balanced_subsample",
        n_jobs=1,
        random_state=RANDOM_SEED
    )))

    models.append(("histgb", HistGradientBoostingClassifier(
        learning_rate=0.05,
        max_depth=6,
        max_iter=800,
        min_samples_leaf=80,
        l2_regularization=0.3,
        random_state=RANDOM_SEED
    )))

    models.append(("logreg", LogisticRegression(
        max_iter=4000,
        class_weight="balanced",
        solver="lbfgs",
        n_jobs=1
    )))

    models.append(("sgd_logloss", SGDClassifier(
        loss="log_loss",
        alpha=5e-5,
        penalty="l2",
        class_weight="balanced",
        max_iter=5000,
        tol=1e-3,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=10,
        random_state=RANDOM_SEED
    )))

    models.append(("xgb_final", "xgb_final"))
    return models


In [12]:
def xgb_train_with_es(
    X_tr, y_tr, X_val, y_val,
    w_tr=None, w_val=None,
    params=None,
    num_boost_round=3000,
    early_stopping_rounds=80,
    verbose_eval=200,
    seed=42
):
    if params is None:
        params = {}

    base_params = {
        "objective": "binary:logistic",
        "eval_metric": "aucpr",
        "tree_method": "hist",
        "seed": seed,
        "max_bin": 256,
    }
    base_params.update(params)

    dtr = xgb.DMatrix(X_tr, label=y_tr, weight=w_tr)
    dval = xgb.DMatrix(X_val, label=y_val, weight=w_val)

    booster = xgb.train(
        params=base_params,
        dtrain=dtr,
        num_boost_round=num_boost_round,
        evals=[(dtr, "train"), (dval, "val")],
        early_stopping_rounds=early_stopping_rounds,
        verbose_eval=verbose_eval
    )

    proba_val = booster.predict(dval, iteration_range=(0, booster.best_iteration + 1))
    pr_auc = float(average_precision_score(y_val, proba_val))
    return booster, pr_auc


def xgb_sample_params(rng: np.random.Generator) -> Dict:
    return {
        "max_depth": int(rng.integers(3, 8)),
        "min_child_weight": float(rng.choice([50, 80, 120, 200, 300])),
        "subsample": float(rng.uniform(0.65, 0.95)),
        "colsample_bytree": float(rng.uniform(0.60, 0.95)),
        "gamma": float(rng.uniform(0.0, 0.6)),
        "lambda": float(rng.uniform(0.5, 4.0)),
        "alpha": float(rng.uniform(0.0, 2.0)),
        "eta": float(rng.uniform(0.02, 0.08)),
    }


def xgb_fit_final_model(X_train, y_train):
    xgb_val_size = 0.15
    xgb_final_val_size = 0.12
    n_trials = 12
    early_stop = 80
    max_rounds = 3000
    extra_rounds = 200

    X_tr, X_val, y_tr, y_val = train_test_split(
        X_train, y_train,
        test_size=xgb_val_size,
        random_state=RANDOM_SEED,
        stratify=y_train
    )

    w_tr = compute_sample_weight(class_weight="balanced", y=y_tr)
    w_val = compute_sample_weight(class_weight="balanced", y=y_val)

    rng = np.random.default_rng(RANDOM_SEED)
    best = {"pr_auc": -1.0, "params": None, "booster": None}

    for _ in range(n_trials):
        params = xgb_sample_params(rng)
        booster, pr_auc = xgb_train_with_es(
            X_tr, y_tr, X_val, y_val,
            w_tr=w_tr, w_val=w_val,
            params=params,
            num_boost_round=max_rounds,
            early_stopping_rounds=early_stop,
            verbose_eval=250,
            seed=RANDOM_SEED
        )
        if pr_auc > best["pr_auc"]:
            best = {"pr_auc": pr_auc, "params": params, "booster": booster}

    X_tr2, X_val2, y_tr2, y_val2 = train_test_split(
        X_train, y_train,
        test_size=xgb_final_val_size,
        random_state=123,
        stratify=y_train
    )

    w_tr2 = compute_sample_weight(class_weight="balanced", y=y_tr2)
    w_val2 = compute_sample_weight(class_weight="balanced", y=y_val2)

    best_params = dict(best["params"])
    best_iter = int(best["booster"].best_iteration)
    num_rounds = max(best_iter + extra_rounds, 800)

    booster_final, pr_auc_val = xgb_train_with_es(
        X_tr2, y_tr2, X_val2, y_val2,
        w_tr=w_tr2, w_val=w_val2,
        params=best_params,
        num_boost_round=num_rounds,
        early_stopping_rounds=early_stop,
        verbose_eval=200,
        seed=RANDOM_SEED
    )

    meta = {
        "best_params": best_params,
        "search_best_iter": best_iter,
        "final_best_iter": int(booster_final.best_iteration),
        "final_pr_auc_val_sklearn": float(pr_auc_val),
        "num_rounds_used": int(num_rounds),
    }
    return booster_final, meta


In [13]:
def fit_eval_save_sklearn(model, name: str, X_train, y_train, X_test, y_test):
    t0 = time.time()
    model.fit(X_train, y_train)
    fit_secs = time.time() - t0

    if hasattr(model, "predict_proba"):
        y_proba = model.predict_proba(X_test)[:, 1]
    else:
        scores = model.decision_function(X_test)
        y_proba = 1.0 / (1.0 + np.exp(-scores))

    row = eval_binary_classifier(name, y_test.to_numpy(), y_proba)
    row["fit_time_sec"] = float(fit_secs)
    row["n_features"] = int(X_train.shape[1])

    path = save_sklearn_model(model, name)
    return row, path


def update_outputs(results: List[Dict], saved: List[Dict]) -> pd.DataFrame:
    leaderboard = pd.DataFrame(results).sort_values(
        ["pr_auc_test", "roc_auc_test", "best_f1_pos"],
        ascending=False
    ).reset_index(drop=True)

    leaderboard.to_csv(OUT_DIR / "leaderboard.csv", index=False, encoding="utf-8")
    dump_json(saved, OUT_DIR / "saved_models.json")
    return leaderboard


def train_one_model(name: str, model, X_train, y_train, X_test, y_test, results, saved) -> pd.DataFrame:
    print("Entrenando:", name)

    if name != "xgb_final":
        row, path = fit_eval_save_sklearn(model, name, X_train, y_train, X_test, y_test)
        results.append(row)
        saved.append({"model": name, "path": path})
        leaderboard = update_outputs(results, saved)
        print("Guardado:", path, "PR-AUC:", round(row["pr_auc_test"], 4), "ROC-AUC:", round(row["roc_auc_test"], 4))
        return leaderboard

    booster_final, meta = xgb_fit_final_model(X_train, y_train)

    dtest = xgb.DMatrix(X_test, label=y_test)
    y_proba = booster_final.predict(dtest, iteration_range=(0, booster_final.best_iteration + 1))

    row = eval_binary_classifier("xgb_final", y_test.to_numpy(), y_proba)
    results.append(row)

    path = save_xgb_booster(booster_final, "xgb_final")
    saved.append({"model": "xgb_final", "path": path})

    dump_json(meta, OUT_DIR / "metadata" / "xgb_meta.json")

    leaderboard = update_outputs(results, saved)
    print("Guardado:", path, "PR-AUC:", round(row["pr_auc_test"], 4), "ROC-AUC:", round(row["roc_auc_test"], 4))
    return leaderboard


In [14]:
results = []
saved = []
models = dict(build_models())


In [15]:
leaderboard = train_one_model(
    "xgb_final",
    "xgb_final",
    X_train, y_train,
    X_test, y_test,
    results, saved
)
leaderboard.head(10)


Entrenando: xgb_final
[0]	train-aucpr:0.65137	val-aucpr:0.64429
[250]	train-aucpr:0.70245	val-aucpr:0.69747
[500]	train-aucpr:0.70625	val-aucpr:0.70032
[750]	train-aucpr:0.70796	val-aucpr:0.70105
[1000]	train-aucpr:0.70927	val-aucpr:0.70146
[1250]	train-aucpr:0.71052	val-aucpr:0.70181
[1500]	train-aucpr:0.71153	val-aucpr:0.70197
[1535]	train-aucpr:0.71165	val-aucpr:0.70196
[0]	train-aucpr:0.65174	val-aucpr:0.64821
[250]	train-aucpr:0.71266	val-aucpr:0.70162
[457]	train-aucpr:0.71787	val-aucpr:0.70198
[0]	train-aucpr:0.64534	val-aucpr:0.64158
[250]	train-aucpr:0.71008	val-aucpr:0.70157
[500]	train-aucpr:0.71525	val-aucpr:0.70263
[732]	train-aucpr:0.71868	val-aucpr:0.70272
[0]	train-aucpr:0.67837	val-aucpr:0.67101
[250]	train-aucpr:0.70650	val-aucpr:0.69858
[500]	train-aucpr:0.71299	val-aucpr:0.70253
[750]	train-aucpr:0.71660	val-aucpr:0.70364
[1000]	train-aucpr:0.71941	val-aucpr:0.70390
[1250]	train-aucpr:0.72194	val-aucpr:0.70407
[1300]	train-aucpr:0.72246	val-aucpr:0.70407
[0]	train-a

,model,roc_auc_test,pr_auc_test,f1_pos@0.5,prec_pos@0.5,rec_pos@0.5,best_thr_f1,best_f1_pos,best_prec_pos,best_rec_pos,support_pos
0,xgb_final,0.698436,0.568677,0.524156,0.464823,0.600852,0.457833,0.526753,0.425684,0.690757,71397


In [40]:
import os
from pathlib import Path

print("Working directory:")
print(Path.cwd())


Working directory:
C:\Users\pmace\Desktop\ropa


In [41]:
from pathlib import Path

DATA_DIR = Path("data/processed")

print("Buscando datos procesados en:")
print(DATA_DIR.resolve())

if not DATA_DIR.exists():
    print("La carpeta data/processed NO existe")
else:
    files = list(DATA_DIR.glob("*"))
    if not files:
        print("La carpeta existe pero está vacía")
    else:
        print("Archivos encontrados:")
        for f in sorted(files):
            print("-", f.name)


Buscando datos procesados en:
C:\Users\pmace\Desktop\ropa\data\processed
Archivos encontrados:
- devoluciones
- df_all2.pkl
- recs_test_small.pkl
- scen_test_small.pkl


In [42]:
from pathlib import Path

MODELS_DIR = Path("modelos/devoluciones")

print("Buscando modelos en:")
print(MODELS_DIR.resolve())

if not MODELS_DIR.exists():
    print("La carpeta modelos/devoluciones NO existe")
else:
    files = list(MODELS_DIR.glob("*"))
    if not files:
        print("La carpeta existe pero está vacía")
    else:
        print("Contenido encontrado:")
        for f in sorted(files):
            print("-", f.name)


Buscando modelos en:
C:\Users\pmace\Desktop\ropa\modelos\devoluciones
Contenido encontrado:
- A1_encoder.joblib
- A1_metadata.json
- A2_encoder.joblib
- A2_metadata.json
- extratrees.joblib
- histgb.joblib
- leaderboard.csv
- logreg.joblib
- metadata
- model_a1_encoder.joblib
- model_a1_metadata.json
- model_a2_encoder.joblib
- model_a2_metadata.json
- model_a_encoder.joblib
- model_a_metadata.json
- rf.joblib
- saved_models.json
- sgd_logloss.joblib
- xgb_A1_multiclass.json
- xgb_A2_rank.json
- xgb_final.json
- xgb_model_a.json
- xgb_model_a1.json
- xgb_model_a1_talla.json
- xgb_model_a1_talla_encoder.joblib
- xgb_model_a1_talla_metadata.json
- xgb_model_a2.json
- xgb_model_a2_talla_ctx.json
- xgb_model_a2_talla_ctx_encoder.joblib
- xgb_model_a2_talla_ctx_metadata.json


# lectura de datos

In [1]:
import pandas as pd
from pathlib import Path

BASE_PATH = Path("data/processed/devoluciones")

X_train = pd.read_parquet(BASE_PATH / "X_train.parquet")
X_test  = pd.read_parquet(BASE_PATH / "X_test.parquet")
y_train = pd.read_parquet(BASE_PATH / "y_train.parquet")
y_test  = pd.read_parquet(BASE_PATH / "y_test.parquet")

train_idx = pd.read_parquet(BASE_PATH / "train_index.parquet")
test_idx  = pd.read_parquet(BASE_PATH / "test_index.parquet")


In [2]:
def quick_overview(df, name):
    print(f"\n=== {name} ===")
    print("Shape:", df.shape)
    print("Columns:")
    for c in df.columns:
        print(f" - {c}")

quick_overview(X_train, "X_train")
quick_overview(X_test, "X_test")
quick_overview(y_train, "y_train")
quick_overview(train_idx, "train_index")



=== X_train ===
Shape: (679083, 83)
Columns:
 - descuento
 - precio_neto
 - coste_bruto
 - margen
 - n_pedidos
 - n_items_comprados
 - altura_cm
 - peso_kg
 - anio_compra
 - mes_compra
 - edad_en_compra
 - antiguedad_cliente_dias
 - en_promocion
 - margen_relativo
 - desajuste_talla
 - desajuste_talla_abs
 - talla_extrema
 - bmi
 - compras_previas_cliente
 - devoluciones_previas_cliente
 - ratio_devoluciones_previas_cliente
 - ventas_previas_producto
 - devoluciones_previas_producto
 - ratio_devoluciones_previas_producto
 - precio_rel_cat
 - missing_provincia_cliente
 - missing_comunidad
 - missing_altura_cm
 - missing_peso_kg
 - missing_edad_en_compra
 - missing_antiguedad_cliente_dias
 - missing_desajuste_talla
 - missing_bmi
 - canal_online
 - comunidad_aragon
 - comunidad_asturias
 - comunidad_baleares
 - comunidad_canarias
 - comunidad_cantabria
 - comunidad_castilla y leon
 - comunidad_castilla-la mancha
 - comunidad_cataluna
 - comunidad_ceuta
 - comunidad_comunidad valenciana


In [3]:
X_train.dtypes.sort_values()


comunidad_cataluna                        bool
categoria_gorra                           bool
categoria_cinturon                        bool
categoria_camiseta                        bool
categoria_camisa                          bool
                                        ...   
ratio_devoluciones_previas_cliente     float64
ratio_devoluciones_previas_producto    float64
precio_rel_cat                         float64
edad_en_compra                         float64
descuento                              float64
Length: 83, dtype: object

In [4]:
pd.DataFrame({
    "column": X_train.columns,
    "dtype": X_train.dtypes.values
})


,column,dtype
0,descuento,float64
1,precio_neto,float64
2,coste_bruto,float64
3,margen,float64
4,n_pedidos,float64
...,...,...
78,talla_onesize,bool
79,talla_s,bool
80,talla_xl,bool
81,talla_xs,bool


In [5]:
possible_id_cols = [
    c for c in X_train.columns 
    if any(k in c.lower() for k in ["item", "ticket", "product", "sku", "cliente", "customer"])
]

possible_id_cols


['n_items_comprados',
 'antiguedad_cliente_dias',
 'compras_previas_cliente',
 'devoluciones_previas_cliente',
 'ratio_devoluciones_previas_cliente',
 'ventas_previas_producto',
 'devoluciones_previas_producto',
 'ratio_devoluciones_previas_producto',
 'missing_provincia_cliente',
 'missing_antiguedad_cliente_dias']

In [6]:
train_idx.head()
test_idx.head()


,fecha_compra,ticket_id,item_id,customer_id,devuelto
0,2025-01-01,T378172,T378172-001,C209219,0
1,2025-01-01,T378172,T378172-002,C209219,0
2,2025-01-01,T378174,T378174-001,C209220,0
3,2025-01-01,T378174,T378174-002,C209220,1
4,2025-01-01,T378176,T378176-001,C209221,0


In [7]:
X_train.describe(include="all").T.head(20)


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
descuento,679083.0,NaN,NaN,NaN,0.045573,0.090225,0.0,0.0,0.0,0.0,0.35
precio_neto,679083.0,NaN,NaN,NaN,56.286818,30.735368,7.8,27.0,50.0,80.75,130.0
coste_bruto,679083.0,NaN,NaN,NaN,26.878926,16.4384,5.0,12.0,22.0,45.0,65.0
margen,679083.0,NaN,NaN,NaN,29.407892,15.104425,2.8,16.0,27.0,38.5,65.0
n_pedidos,679083.0,NaN,NaN,NaN,2.60113,3.080691,0.0,1.0,2.0,4.0,25.0
n_items_comprados,679083.0,NaN,NaN,NaN,4.16709,5.173268,0.0,1.0,2.0,5.0,49.0
altura_cm,679083.0,NaN,NaN,NaN,177.098089,7.508915,150.1,172.9,176.6,181.0,209.8
peso_kg,679083.0,NaN,NaN,NaN,79.884767,13.151691,40.0,72.3,78.8,86.1,144.0
anio_compra,679083.0,NaN,NaN,NaN,2022.528028,1.609969,2017.0,2022.0,2023.0,2024.0,2025.0
mes_compra,679083.0,NaN,NaN,NaN,6.95885,3.510715,1.0,4.0,7.0,10.0,12.0
